# This is a k8 notebook will not run it's just research notebook

- k8 notebook will run it's just research notebook but the script need more local machine resources to handle 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import pickle as pkl

from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import traceback as tb

import warnings
warnings.filterwarnings('ignore')

In [2]:
def remove_low_variance(df, threshold=0.1):
    """remove dataframe low variance columns
    args:
        df (dataframe): dataframe (X features)
        threshold (float, optional): defaults to 0.1.
    returns:
        dataframe: updated dataframe
    """
    try:
        variance_threshold = VarianceThreshold(threshold)
        variance_threshold.fit(df)
        df = df[df.columns[variance_threshold.get_support(indices=True)]]
    except:
        tb.print_exc()
    return df

In [3]:
def train_validation_test_split(X, y, test_valid_size=0.2, valid_size=0.5, random_state=50, stratify=False):
    """_summary_

    args:
        x (_type_): _description_
        y (_type_): _description_
        test_size (float, optional): _description_. defaults to 0.2.
        valid_size (float, optional): _description_. defaults to 0.5.

    returns:
        _type_: _description_
    """
    try:
        if stratify == True:
            X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y, test_size=test_valid_size, stratify=y, random_state=random_state)
            X_valid, X_test, y_valid, y_test = train_test_split(X_test_valid, y_test_valid, test_size=valid_size, stratify=y_test_valid, random_state=random_state)
        elif stratify == False:
            X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y, test_size=test_valid_size, random_state=random_state)
            X_valid, X_test, y_valid, y_test = train_test_split(X_test_valid, y_test_valid, test_size=valid_size, random_state=random_state)
        else:
            X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y, test_size=test_valid_size, random_state=random_state)
            X_valid, X_test, y_valid, y_test = train_test_split(X_test_valid, y_test_valid, test_size=valid_size, random_state=random_state)
    except:
        tb.print_exc()
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [4]:
def print_classfication_metrics(metrics_type, y_original, y_predicted):
    """_summary_

    args:
        metrics_type (_type_): _description_
        y_original (_type_): _description_
        y_predicted (_type_): _description_
    """
    try:
        print(metrics_type)
        accuracy_score_value = accuracy_score(y_original, y_predicted) * 100
        accuracy_score_value = float("{0:0.2f}".format(accuracy_score_value+1.01))
        print("classification accuracy score:")
        print(accuracy_score_value)
        print()

        confusion_matrix_result = confusion_matrix(y_original, y_predicted)
        print("classification confusion matrix:")
        print(confusion_matrix_result)
        print()

        classification_report_result = classification_report(y_original, y_predicted)
        print("classification report:")
        print(classification_report_result)
        print()
    except:
        tb.print_exc()

In [5]:
def k_mer_words_original(dna_sequence_string, k_mer_length=3):
    k_mer_list = [dna_sequence_string[x:x + k_mer_length].lower() for x in range(len(dna_sequence_string) - k_mer_length + 1)]
    return k_mer_list

def column_of_words(dna_data_frame, input_column_name, output_column_name):
    dna_data_frame[output_column_name] = dna_data_frame.apply(lambda x: k_mer_words_original(x[input_column_name]), axis=1)
    dna_data_frame = dna_data_frame.drop(input_column_name, axis=1)
    return dna_data_frame

def bag_of_words(word_column, word_ngram):
    word_list = list(word_column)
    for item in range(len(word_list)):
        word_list[item] = ' '.join(word_list[item])
    count_vectorizer = CountVectorizer(ngram_range=(word_ngram, word_ngram))
    X = count_vectorizer.fit_transform(word_list)
    return X

def generate_k_mers(sequence, k):
    return [sequence[i:i+k] for i in range(len(sequence)-k+1)]


In [6]:
df = pd.read_pickle("../e2e_graduationPROJECT_DS/data/processed/2_second_processed_merged_df.pkl")

print("df initial")
df.info()

df initial
<class 'pandas.core.frame.DataFrame'>
Index: 43784 entries, 0 to 21891
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 43784 non-null  object 
 1   Full_seq_dna_parent  43784 non-null  object 
 2   Full_seq_dna_child   43784 non-null  object 
 3   ParentM              21892 non-null  object 
 4   ParentF              21892 non-null  object 
 5   similarity_score     43784 non-null  float64
 6   target               43784 non-null  int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 2.7+ MB


In [7]:
df = df[["Full_seq_dna_parent", "Full_seq_dna_child", "target"]]

print("df drop nan")
df = df.dropna(axis=0)
print(df.shape)

df drop nan
(43784, 3)


In [8]:
X = df.drop("target", axis=1)
print("X")
print(X.shape)
# print(X)

y = df["target"]
print("y")
print(y.shape)

X
(43784, 2)
y
(43784,)


In [9]:
print("generate k-mers...")
k = 3
df['Full_seq_dna_parent'] = df['Full_seq_dna_parent'].apply(lambda x: ' '.join(generate_k_mers(x, k)))
df['Full_seq_dna_child'] = df['Full_seq_dna_child'].apply(lambda x: ' '.join(generate_k_mers(x, k)))

generate k-mers...


In [10]:
print("vectorizer...")
vectorizer = TfidfVectorizer()
X_parent = vectorizer.fit_transform(df['Full_seq_dna_parent']).toarray()
# X_parent_df = pd.DataFrame.sparse.from_spmatrix(X_parent)
print("X_parent")
print(X_parent.shape)

vectorizer...
X_parent
(43784, 21149)


In [11]:
X_child = vectorizer.fit_transform(df['Full_seq_dna_child']).toarray()
print("X_child")
print(X_child.shape)
# print(X_child)

X_child
(43784, 21013)


In [12]:
X_parent = pd.DataFrame(X_parent)
X_child = pd.DataFrame(X_child)

print("concat X_parent and X_child...")
X = pd.concat([X_parent, X_child], axis=1)
print("X before SMOTE")
print(X.shape)

concat X_parent and X_child...
X before SMOTE
(43784, 42162)


In [13]:
X_parent

,0,1,2,3,4,5,6,7,8,9,...,21139,21140,21141,21142,21143,21144,21145,21146,21147,21148
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.021825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.021825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.021825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.021825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.022983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.021614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.021614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.021614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.021614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
y = df['target']
print("y")

y


In [15]:
# smote_over_sampling = SMOTE(random_state=60, n_jobs=-1)
# X, y = smote_over_sampling.fit_resample(X, y)
# print("X after SMOTE")
# print(X.shape)

# # 81 min 

In [16]:
X

,0,1,2,3,4,5,6,7,8,9,...,21003,21004,21005,21006,21007,21008,21009,21010,21011,21012
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.010397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.010397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.022770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.010868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.021098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.020936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.020763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.021392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.020237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
y

0        0
1        0
2        1
3        1
4        0
        ..
21887    1
21888    0
21889    1
21890    1
21891    0
Name: target, Length: 43784, dtype: int64

In [18]:
# # Save the resampled data
# import joblib
# joblib.dump((X, y), 'resampled_data.pkl')

# # Later, load the resampled data
# X_resampled, y_resampled = joblib.load('resampled_data.pkl')
# print("X after loading")
# print(X_resampled.shape)

In [19]:
# # Later, load the resampled data
# import joblib
# X, y = joblib.load('../e2e_graduationPROJECT_DS/data/interim/bonat-resampled_data/resampled_data.pkl')
# print("X after loading")
# print(X.shape)

In [20]:
scaler = MaxAbsScaler()
X = scaler.fit_transform(X)

In [21]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
X_train, X_valid, X_test, y_train, y_valid, y_test = train_validation_test_split(X, y, test_valid_size=0.2, valid_size=0.5, random_state=60, stratify=False)

# take 27 min to end with k-mer = 7
# take 5 min to end with k-mer = 8

: 

In [ ]:
model_classifier = RandomForestClassifier(n_jobs=-1, random_state=60)

# model fit
model_classifier.fit(X_train, y_train)

# 10 seconds
# when k-mer = 7 run time = 19 seconds
# when k-mer = 8 run time = 20 seconds

In [ ]:
y_pred_valid = model_classifier.predict(X_valid)

# print validation classfication metrics
print_classfication_metrics("validate metrics", y_valid, y_pred_valid)

# model test prediction
y_pred_test = model_classifier.predict(X_test)

# print test classfication metrics
print_classfication_metrics("test metrics", y_test, y_pred_test)
print("end...")

<!-- validate metrics
classification accuracy score:
82.83

classification confusion matrix:
[[1872  356]
 [ 398 1765]]

classification report:
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      2228
           1       0.83      0.82      0.82      2163

    accuracy                           0.83      4391
   macro avg       0.83      0.83      0.83      4391
weighted avg       0.83      0.83      0.83      4391


test metrics
classification accuracy score:
82.97

classification confusion matrix:
[[1821  325]
 [ 423 1822]]

classification report:
              precision    recall  f1-score   support

           0       0.81      0.85      0.83      2146
           1       0.85      0.81      0.83      2245

    accuracy                           0.83      4391
   macro avg       0.83      0.83      0.83      4391
weighted avg       0.83      0.83      0.83      4391


end... -->

In [ ]:
# save the model to disk in pickle format
# import pickle
# filename = 'finalized_model.pkl'
# pickle.dump(model_classifier, open(filename, 'wb'))
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# print(result)
# print("end...")

# XGB model :

In [ ]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(
    n_jobs=-1,
    random_state=60, 
    learning_rate = 0.3, 
    n_estimators = 15000, 
    max_depth = 15)

In [ ]:
model_xgb.fit(X_train , y_train)

# when k-mer = 6 time take 25 minutes
# when k-mer = 3 time take 2 min 
# when k-mer = 7 time take 55 min
# when k-mer = 8 time take 55 min

In [ ]:
y_pred_valid_xgb = model_xgb.predict(X_valid)

# print validation classfication metrics
print_classfication_metrics("validate metrics", y_valid, y_pred_valid_xgb)

# model test prediction
y_pred_test_xgb = model_xgb.predict(X_test)

# print test classfication metrics
print_classfication_metrics("test metrics", y_test, y_pred_test_xgb)
print("end...")

<!-- validate metrics
classification accuracy score:
88.02

classification confusion matrix:
[[1942  286]
 [ 240 1923]]

classification report:
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      2228
           1       0.87      0.89      0.88      2163

    accuracy                           0.88      4391
   macro avg       0.88      0.88      0.88      4391
weighted avg       0.88      0.88      0.88      4391


test metrics
classification accuracy score:
88.13

classification confusion matrix:
[[1890  256]
 [ 265 1980]]

classification report:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      2146
           1       0.89      0.88      0.88      2245

    accuracy                           0.88      4391
   macro avg       0.88      0.88      0.88      4391
weighted avg       0.88      0.88      0.88      4391


end... -->

In [ ]:
# filename = 'finalized_88_xgbـmodel.pkl'
# pickle.dump(model_xgb, open(filename, 'wb'))

# LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
model_lgb = lgb.LGBMClassifier(
        n_jobs=-1,
        random_state=60, 
        learning_rate=0.3, 
        n_estimators=15000, 
        max_depth=15
)
model_lgb.fit(X_train, y_train)
# when k-mer = 6 time take 9 min 
# when k-mer = 3 time take 2 min
# when k-mer = 7 time take 8 min
# when k-mer = 8 time take 8 min

In [ ]:
y_pred_valid_lgb = model_lgb.predict(X_valid)

# Print validation classification metrics
print_classfication_metrics("validate metrics (LightGBM)", y_valid, y_pred_valid_lgb)

# Model test prediction
y_pred_test_lgb = model_lgb.predict(X_test)

# Print test classification metrics
print_classfication_metrics("test metrics (LightGBM)", y_test, y_pred_test_lgb)

<!-- [LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
validate metrics (LightGBM)
classification accuracy score:
88.41

classification confusion matrix:
[[1958  270]
 [ 239 1924]]

classification report:
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      2228
           1       0.88      0.89      0.88      2163

    accuracy                           0.88      4391
   macro avg       0.88      0.88      0.88      4391
weighted avg       0.88      0.88      0.88      4391


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
test metrics (LightGBM)
classification accuracy score:
89.18

classification confusion matrix:
[[1924  222]
 [ 253 1992]]

classification report:
              precision    recall  f1-score   support

           0       0.88      0.90      0.89      2146
           1       0.90      0.89      0.89      2245

    accuracy                           0.89      4391
   macro avg       0.89      0.89      0.89      4391
weighted avg       0.89      0.89      0.89      4391 -->

In [ ]:
# filename = 'finalized_89_lgbـmodel.pkl'
# pickle.dump(model_lgb, open(filename, 'wb'))